In [1]:
from zipfile import ZipFile

with ZipFile('/fs/ess/PGS0333/BA_64061_KSU/data/dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('/users/PGS0333/rishitha/AML_Data')

In [2]:
import os
os.getcwd()

'/users/PGS0333/rishitha/osc_classes/BA_64061_KSU'

In [3]:
os.chdir('/users/PGS0333/rishitha/AML_Data')

In [4]:
# Unzipping dogs-vs-cats dataset file
!unzip -qq dogs-vs-cats.zip

unzip:  cannot find or open dogs-vs-cats.zip, dogs-vs-cats.zip.zip or dogs-vs-cats.zip.ZIP.


In [5]:
# Unzipping train sample 
!unzip -o -qq train.zip

^C


In [6]:
# Creating directories and assiging images to training, validation and test directories
import os, shutil, pathlib

shutil.rmtree("cats_vs_dogs_small")

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [7]:
#Building the model and running the model summary 

from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()
     

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 128)       73856 

In [8]:
# Configuration of the model
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [9]:
# Declaring the image size and batch size to read the images from train. validation and test directories
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)
     

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [10]:
# Using the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 88s 1s/step - loss: 0.7205 - accuracy: 0.5280 - val_loss: 0.6910 - val_accuracy: 0.5030
Epoch 2/30
63/63 [==============================] - 86s 1s/step - loss: 0.7094 - accuracy: 0.5415 - val_loss: 0.7159 - val_accuracy: 0.5010
Epoch 3/30
63/63 [==============================] - 85s 1s/step - loss: 0.7054 - accuracy: 0.5815 - val_loss: 0.6530 - val_accuracy: 0.5870
Epoch 4/30
63/63 [==============================] - 85s 1s/step - loss: 0.6762 - accuracy: 0.6175 - val_loss: 0.6683 - val_accuracy: 0.5610
Epoch 5/30
63/63 [==============================] - 85s 1s/step - loss: 0.6146 - accuracy: 0.6445 - val_loss: 0.6684 - val_accuracy: 0.6220
Epoch 6/30
63/63 [==============================] - 85s 1s/step - loss: 0.5890 - accuracy: 0.6995 - val_loss: 0.7382 - val_accuracy: 0.5980
Epoch 7/30
63/63 [==============================] - 85s 1s/step - loss: 0.5595 - accuracy: 0.7195 - val_loss: 0.6750 - val_accuracy: 0.6550
Epoch 8/30
63/63 [==

In [10]:
# Testing the model 
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")
     


63/63 [==============================] - 21s 327ms/step - loss: 0.0967 - accuracy: 0.9630
Test accuracy: 0.963


In [11]:
# Declaring Data Augumentation
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

In [12]:
# Building the model and configuing it 
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [14]:

   # Using the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch_with_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=40,
    validation_data=validation_dataset,
    callbacks=callbacks)  

Epoch 1/40
63/63 [==============================] - 95s 1s/step - loss: 0.7252 - accuracy: 0.5245 - val_loss: 0.6923 - val_accuracy: 0.5310
Epoch 2/40
63/63 [==============================] - 92s 1s/step - loss: 0.7039 - accuracy: 0.5120 - val_loss: 0.6813 - val_accuracy: 0.5590
Epoch 3/40
63/63 [==============================] - 92s 1s/step - loss: 0.6948 - accuracy: 0.5435 - val_loss: 0.6783 - val_accuracy: 0.5960
Epoch 4/40
63/63 [==============================] - 92s 1s/step - loss: 0.6812 - accuracy: 0.5905 - val_loss: 0.6632 - val_accuracy: 0.6010
Epoch 5/40
63/63 [==============================] - 92s 1s/step - loss: 0.6575 - accuracy: 0.6220 - val_loss: 0.6427 - val_accuracy: 0.6370
Epoch 6/40
63/63 [==============================] - 92s 1s/step - loss: 0.6478 - accuracy: 0.6400 - val_loss: 0.7441 - val_accuracy: 0.5470
Epoch 7/40
63/63 [==============================] - 92s 1s/step - loss: 0.6387 - accuracy: 0.6510 - val_loss: 0.6171 - val_accuracy: 0.6560
Epoch 8/40
63/63 [==

In [13]:
test_model = keras.models.load_model(
    "convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")
     

63/63 [==============================] - 21s 327ms/step - loss: 0.4701 - accuracy: 0.8045
Test accuracy: 0.804


In [14]:
# Creating directories and assiging images to training, validation and test directories
# Increasing the training sample from 1000 to 2000
import os
import shutil
import pathlib
shutil.rmtree("cats_vs_dogs_small_IncreasedTrainSample")
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_IncreasedTrainSample")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

# Training has 2000 samples, test has 1000 samples, and validation has 500 samples
make_subset("train", start_index=0, end_index=2000)
make_subset("validation", start_index=2000, end_index=2500)
make_subset("test", start_index=2500, end_index=3500)

In [15]:
# Building the model
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [16]:
# Configuring the model 
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
     

In [41]:
# Using the callbacks function to monitor validation loss and running the model
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/20
63/63 [==============================] - 88s 1s/step - loss: 0.7456 - accuracy: 0.5110 - val_loss: 0.6885 - val_accuracy: 0.5470
Epoch 2/20
63/63 [==============================] - 86s 1s/step - loss: 0.7061 - accuracy: 0.5625 - val_loss: 0.6680 - val_accuracy: 0.5950
Epoch 3/20
63/63 [==============================] - 87s 1s/step - loss: 0.6763 - accuracy: 0.5940 - val_loss: 0.6404 - val_accuracy: 0.6160
Epoch 4/20
63/63 [==============================] - 86s 1s/step - loss: 0.6404 - accuracy: 0.6445 - val_loss: 0.6009 - val_accuracy: 0.6810
Epoch 5/20
63/63 [==============================] - 86s 1s/step - loss: 0.6114 - accuracy: 0.6835 - val_loss: 0.6029 - val_accuracy: 0.6890
Epoch 6/20
63/63 [==============================] - 86s 1s/step - loss: 0.5697 - accuracy: 0.7105 - val_loss: 0.5398 - val_accuracy: 0.7200
Epoch 7/20
63/63 [==============================] - 86s 1s/step - loss: 0.5191 - accuracy: 0.7365 - val_loss: 0.8860 - val_accuracy: 0.6280
Epoch 8/20
63/63 [==

In [17]:
# Testing the model 
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")
     

63/63 [==============================] - 21s 331ms/step - loss: 0.0967 - accuracy: 0.9630
Test accuracy: 0.963


In [18]:
# Creating directories and assiging images to training, validation and test directories
# Increasing the training sample from 1000 to 2000
import os
import shutil
import pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small_OptimalTrainSamples1")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir,exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

# Training has 1500 samples, test has 1000 samples, and validation has 500 samples
# make_subset("train", start_index=0, end_index=3500)
# make_subset("validation", start_index=2500, end_index=3000)
# make_subset("test", start_index=3000, end_index=4000)


In [19]:
# Building the model
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
     

In [20]:
# Configuring the model 
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator


# Train the model with varying training sample sizes
sample_sizes = [3500,4000,4500,5000]
history_dict = []
for size in sample_sizes:
    # Set up the training subset
    make_subset("temp_train", start_index=1500, end_index=size)
    make_subset("validation", start_index=size, end_index=size+500)
    make_subset("test", start_index=size+500, end_index=size+1500)
    train_dataset = image_dataset_from_directory(
      new_base_dir / "temp_train", 
      image_size=(180, 180), 
      batch_size=20)
    # Using the callbacks function to monitor validation loss and running the model
  
    callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")]

    history = model.fit(
      train_dataset,
      epochs=20,
      validation_data=validation_dataset,
      callbacks=callbacks)
    
    test_model = keras.models.load_model("convnet_from_scratch.keras")
    test_loss, test_acc = test_model.evaluate(test_dataset)
    history_dict.append(test_acc)
    print(f"Test accuracy: {test_acc:.3f}")  
 

Found 4000 files belonging to 2 classes.
Epoch 1/20
200/200 [==============================] - 175s 869ms/step - loss: 0.7034 - accuracy: 0.5362 - val_loss: 0.7061 - val_accuracy: 0.5180
Epoch 2/20
200/200 [==============================] - 170s 852ms/step - loss: 0.6654 - accuracy: 0.6070 - val_loss: 0.6170 - val_accuracy: 0.6560
Epoch 3/20
200/200 [==============================] - 170s 851ms/step - loss: 0.6156 - accuracy: 0.6622 - val_loss: 0.5946 - val_accuracy: 0.6810
Epoch 4/20
200/200 [==============================] - 170s 849ms/step - loss: 0.5899 - accuracy: 0.6883 - val_loss: 0.5980 - val_accuracy: 0.6820
Epoch 5/20
200/200 [==============================] - 170s 852ms/step - loss: 0.5477 - accuracy: 0.7240 - val_loss: 0.5510 - val_accuracy: 0.7210
Epoch 6/20
200/200 [==============================] - 171s 853ms/step - loss: 0.5013 - accuracy: 0.7573 - val_loss: 0.5049 - val_accuracy: 0.7600
Epoch 7/20
200/200 [==============================] - 170s 852ms/step - loss: 0.437

In [21]:
import matplotlib.pyplot as plt
plt.plot(sample_sizes, history_dict, 'bo-')
plt.xlabel('Number of training samples')
plt.ylabel('Test accuracy')
plt.title('Effect of training sample size on CNN performance')
plt.show()

NameError: name 'sample_sizes' is not defined